### Test sentiment ML 2

In [9]:
from parse import *
from word_tools import *


In [1]:
from word_tools import make_bagofwords

nb_words = 10000

bow = make_bagofwords("data/corpus.data", 10000)
print(bow[-50:])

['contre', 'être', 'européens', 'ont', 'tous', 'grand', 'même', 'politique', 'bien', 'français', 'président', 'lui', 'ils', 'faire', 'tout', 'comme', 'sont', 'raffarin', 'gilets', 'jaunes', 'renaissance', 'tribune', 'http', 'nous', 'fait', 'mais', 'via', 'européennes', 'france', 'par', 'son', 'plus', 'européenne', 'avec', 'sur', 'aux', 'europe', 'vous', 'dans', 'emmanuel', 'qui', 'pas', 'que', 'une', 'des', 'pour', 'est', 'les', '', 'macron']


In [3]:
from word_tools import vectorize_tweets
vect = vectorize_tweets("data/corpus.data", bow)

In [4]:
import numpy as np
keys = [k for k in vect]

input_len = np.shape(vect[keys[0]]['vectorized'])[0]

print(np.shape(vect[keys[0]]['vectorized']))

(49,)


In [5]:

def extract_data(vectsdict):
    import numpy as np
    vects = []
    labels = []
    
    for key in vectsdict :
        if not np.all(vectsdict[key]['label'] == 0) :
            vects.append(vectsdict[key]['vectorized'])
            labels.append(vectsdict[key]['label'])
    return np.array(vects), np.array(labels)

def decode_output(out_array) :
    if out_array[0] == np.max(out_array) :
        return "irr", np.max(out_array)
    elif out_array[1] == np.max(out_array) :
        return "neg", np.max(out_array)
    elif out_array[2] == np.max(out_array) :
        return "neu",np.max(out_array)
    elif out_array[3] == np.max(out_array) :
        return "pos",np.max(out_array)
    else :
        return "err",0
    
data, labels = extract_data(vect)


percentage_train = 0.8
borne = int(percentage_train*len(data))
X_train = data[:borne]
Y_train = labels[:borne]
X_test = data[borne:]
Y_test = labels[borne:]



[9989 9999 9744 9992 9808 9982    0 8347 7170 9975 9892 9991 4365 9997
 8083 7171 9990    0 9984 9942 6223 9892 9991 7172 9751 9901 9734    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0]


In [6]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.embeddings import Embedding

# define the model
model = Sequential()
model.add(Embedding(input_dim=nb_words, output_dim=8, input_length=input_len))
model.add(Flatten())
model.add(Dense(4, activation='sigmoid'))

# compile the model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# print(model.summary())
# batch = 50
batch = 100
model.fit(X_train, Y_train, epochs=70, batch_size=batch, verbose=1)



Using TensorFlow backend.


Epoch 1/70
3998/3998 [==============================] - 0s 57us/step - loss: 0.6521 - acc: 0.7067
Epoch 2/70
3998/3998 [==============================] - 0s 53us/step - loss: 0.5110 - acc: 0.7669
Epoch 3/70
3998/3998 [==============================] - 0s 47us/step - loss: 0.4559 - acc: 0.7842
Epoch 4/70
3998/3998 [==============================] - 0s 32us/step - loss: 0.4395 - acc: 0.7999
Epoch 5/70
3998/3998 [==============================] - 0s 21us/step - loss: 0.4200 - acc: 0.8189
Epoch 6/70
3998/3998 [==============================] - 0s 28us/step - loss: 0.3983 - acc: 0.8344
Epoch 7/70
3998/3998 [==============================] - 0s 29us/step - loss: 0.3767 - acc: 0.8446
Epoch 8/70
3998/3998 [==============================] - 0s 29us/step - loss: 0.3565 - acc: 0.8569
Epoch 9/70
3998/3998 [==============================] - 0s 30us/step - loss: 0.3380 - acc: 0.8674
Epoch 10/70
3998/3998 [==============================] - 0s 26us/step - loss: 0.3200 - acc: 0.8774
Epoch 11/70
3998/39

In [7]:
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
# loss, accuracy = model.evaluate(X_train, Y_train)
print(accuracy)

0.80425


In [11]:
test = "Macron n'est pas si mal"

test_vect = vectorize_tweet(test, "pos", bow, input_len)
v = np.array(test_vect['vectorized'])
v = v.reshape((1,input_len))

res = model.predict([v])[0]
out = decode_output(res)
print("tweet is evaluated " + out[0])

tweet is evaluated irr


In [10]:
from parse import sample_data
testset = sample_data("tw_db_prepared.data", 5000)


In [13]:
# from note import write_emotion
neu=[]
irr=[]
positives=[]
negatives=[]

for tw in testset :
    vec = vectorize_tweet(tw,'???', bow, input_len)
#     print(vec['text'])
    res = model.predict(vec['vectorized'].reshape((1,input_len)))[0]
    out = decode_output(res)
    if out[0] == "pos" :
        positives.append(tw)
    elif out[0] == "neu" :
        neu.append(tw)
    elif out[0] == "neg" :
        negatives.append(tw)
    elif out[0] == "irr" :
        irr.append(tw)
#     print("tweet is evaluated " + out[0] + "\n\n")

print(len(positives))

83


In [14]:
for p in positives :
    print(p)

merci retweet selon sondage odoxa dentsu consulting une majorité français rejettent les manifestations hebdomadaires des gilets jaunes image président république améliore vous supportez toujours les gilets jaunes giletsjaunes odoxa macron
merci psa merci macron cet appel generosite est plus proche concessionnaire veux une peugeot 
chez les petites phrases macron sont plupart temps justes sont accord vent tourne plus plus 
les sont aussi responsables pic pollution aux particules fines les sont responsables tout est nouvelle règle emmanuel macron 
acte david tabassé par policier suis réveillé dans une mare sang via sérieux comment font ces flics pour bien vivre jusqu taper une personne invalide vénération absolue pour macron cinglés 
alex vous avez pas vous excusez vous expliquer vous faites second degré les supporters pouvoir place macron lui même est choqué vous demande des comptes directement indirectement bref vous avez comprendre vous mêmes terrible pour france
jean françois copé es

In [15]:
import sys
print(sys.version)

3.6.7 (default, Nov 30 2018, 15:49:23) 
[GCC 8.2.1 20180831]
